In [2]:
import os
os.chdir("../")

In [24]:
pwd

'c:\\Ineuron\\DeepCnnClassifier'

In [39]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
   root_dir : Path
   base_model_path : Path
   updated_base_model_path : Path
   params_img_size: list
   params_learning_rate: float
   params_include_top: bool
   params_weights: str
   params_classes: int

In [40]:
from src.deepclassifier.constants import *

In [41]:
from src.deepclassifier.utils import read_yaml, create_directories

In [42]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            
            self.c = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.c.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
          c = self.c.prepare_base_model

          create_directories([c.root_dir])

          prepare_base_model_config = PrepareBaseModelConfig(
                root_dir = Path(c.root_dir),
                base_model_path = Path(c.base_model_path),
                updated_base_model_path = Path(c.updated_base_model_path),
                params_img_size= self.params.IMAGE_SIZE,
                params_learning_rate = self.params.LEARNING_RATE,
                params_include_top = self.params.INCLUDE_TOP,
                params_weights= self.params.WEIGHTS,
                params_classes = self.params.CLASSES
          )

          return prepare_base_model_config
    

In [50]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [51]:
class PrepareBaseModel:
    def __init__(self, c:PrepareBaseModelConfig) -> None:
        self.c=c

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.c.params_img_size,
            weights = self.c.params_weights,
            include_top=self.c.params_include_top
        )

        #base_model_path = Path(self.c.base_model_path)
        self.save_model(path=self.c.base_model_path, model=self.model)
    
    @staticmethod
    def _prepare_full_model(model,classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable=False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable= False     

        flatten_in = tf.keras.layers.Flatten(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="Softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction
        )

        full_model.compile(
            optimizers = tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ["accuracy"]

        )         

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model = self.model,
            classes = self.c.params_classes,
            freeze_all =True,
            freeze_till = None,
            learning_rate = self.c.params_learning_rate

        )

        self.save_model(path=self.c.updated_base_model_path,model=self.full_model)
    
    @staticmethod
    def save_model(path: Path, model:tf.keras.Model):
        model.save(path)
        

In [57]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class PrepareBaseModel:
    def __init__(self, c: PrepareBaseModelConfig):
        self.c = c

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.c.params_img_size,
            weights=self.c.params_weights,
            include_top=self.c.params_include_top
        )

        self.save_model(path=self.c.base_model_path, model=self.model)


    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.c.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.c.params_learning_rate
        )

        self.save_model(path=self.c.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [58]:
try:
    c = ConfigurationManager()
    prepare_base_model_config = c.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(c= prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
    #prepare_base_model.save_model()
except Exception as e:
    raise e    

[2024-05-08 19:57:48,576: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2024-05-08 19:57:48,581: INFO: common]: yaml file: params.yaml loaded successfully
[2024-05-08 19:57:48,583: INFO: common]: Created directory at : artifacts
[2024-05-08 19:57:48,584: INFO: common]: Created directory at : artifacts/prepare_base_model
[2024-05-08 19:57:49,132: WARNING: saving_utils]: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224,

In [13]:
class Square:
    def __init__(self,x):
        self.x=x

    def __call__(self):
        return self.x**2
    
#sq = Square(4)
sq = Square(4)()

In [14]:
#sq()
sq

16

In [18]:
class Power:
    def __init__(self,x):
        self.x=x

    def __call__(self,factor):
        return self.x**factor
    
#sq = Square(4)
res = Power(4)(3)

In [19]:
res

64

In [20]:
class Power:
    def __init__(self,x):
        self.x=x

    def __call__(self,factor):
        return self.x**factor
    
    def square(self):
        return self.x**2
#sq = Square(4)
res = Power(4)

In [22]:
res = Power(4)
res.square()

16